In [1]:
#Project: 멋진 챗봇 만들기

# NLTK를 활용한 BLEU Score
참고 블루 스코어 설명 https://donghwa-kim.github.io/BLEU.html
BLEU(Bilingual Evaluation Understudy)score란 성과지표로 데이터의 X가 순서정보를 가진 단어들(문장)로 이루어져 있고, y 또한 단어들의 시리즈(문장)로 이루어진 경우에 사용되며, 번역을 하는 모델에 주로 사용된다.

# project 챗봇 만들기

In [2]:
import numpy as np
import tensorflow as tf

import re
import io
import time
import random

from sklearn.model_selection import train_test_split

print(tf.__version__)

2.4.1


In [3]:
import os
import pandas as pd
from pandas import DataFrame

data_dir = os.getenv('HOME')+'/aiffel/songys_chatbot/Chatbot_data-master'
data = pd.read_csv("/home/aiffel-dj57/aiffel/songys_chatbot/Chatbot_data-master/ChatbotData .csv")
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [4]:
#데이터 다운로드
#읽어 온 데이터의 질문과 답변을 각각 questions, answers 변수에 나눠서 저장하세요!

df = pd.DataFrame(data) ## 데이터프레임 생성
questions = df['Q']
answers = df['A']

In [5]:
questions

0                         12시 땡!
1                    1지망 학교 떨어졌어
2                   3박4일 놀러가고 싶다
3                3박4일 정도 놀러가고 싶다
4                        PPL 심하네
                  ...           
11818             훔쳐보는 것도 눈치 보임.
11819             훔쳐보는 것도 눈치 보임.
11820                흑기사 해주는 짝남.
11821    힘든 연애 좋은 연애라는게 무슨 차이일까?
11822                 힘들어서 결혼할까봐
Name: Q, Length: 11823, dtype: object

In [6]:
answers

0                      하루가 또 가네요.
1                       위로해 드립니다.
2                     여행은 언제나 좋죠.
3                     여행은 언제나 좋죠.
4                      눈살이 찌푸려지죠.
                   ...           
11818          티가 나니까 눈치가 보이는 거죠!
11819               훔쳐보는 거 티나나봐요.
11820                      설렜겠어요.
11821    잘 헤어질 수 있는 사이 여부인 거 같아요.
11822          도피성 결혼은 하지 않길 바라요.
Name: A, Length: 11823, dtype: object

In [7]:
# 데이터 정제
#영문자의 경우, 모두 소문자로 변환합니다.
#영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 정규식을 활용하여 모두 제거합니다.


def preprocess_sentence(sentence, s_token=False, e_token=False):
    sentence = sentence.lower().strip()
    
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^ㄱ-ㅣ가-힣a-zA-Z0-9?.!,]+", " ", sentence)

    sentence = sentence.strip()

    if s_token:
        sentence = '<start> ' + sentence

    if e_token:
        sentence += ' <end>'
    
    return sentence

# 토큰화
토큰화에는 KoNLPy의 mecab 클래스를 사용합니다.

아래 조건을 만족하는 build_corpus() 함수를 구현하세요!

1. 소스 문장 데이터와 타겟 문장 데이터를 입력으로 받습니다.
2. 데이터를 앞서 정의한 preprocess_sentence() 함수로 정제하고, 토큰화합니다. 토큰화는 전달받은 토크나이즈 함수를 사용합니다. 이번엔 mecab.morphs 함수를 전달하시면 됩니다.
3. 토큰의 개수가 일정 길이 이상인 문장은 데이터에서 제외합니다.중복되는 문장은 데이터에서 제외합니다. 소스 : 타겟 쌍을 비교하지 않고 소스는 소스대로 타겟은 타겟대로 검사합니다. 중복 쌍이 흐트러지지 않도록 유의하세요!
4. 구현한 함수를 활용하여 questions 와 answers 를 각각 que_corpus , ans_corpus 에 토큰화하여 저장합니다.

In [8]:
#q_list = df.Q.str.split('_').str[0].tolist()
#print(q_list[:10])
#a_list = df.A.str.split('_').str[0].tolist() #리스트로 변환해줍니다. 될지는 모르겠어요 
#print(a_list[:10])

#이런 식으로 리스트 형태로 만들어 아래 코드에 하나하나 넣어줬더니 리스트 전체를 도는데 시간이 오래걸려서 에러가 생겼다. 이 방법은 아닌 것 같다

In [9]:
from konlpy.tag import Mecab
tokenizer = Mecab()

In [10]:
def build_corpus(src, tgt, tokenizer, max_len=50):

    pre_src = [preprocess_sentence(x) for x in src]
    pre_tgt = [preprocess_sentence(x) for x in tgt]
    
    #중복제거
    src = []
    tgt = []
    
    for so,tar in zip(pre_src, pre_tgt):
        if so in src :
            continue
        if tar in tgt :
            continue
        src.append(so)
        tgt.append(tar)
    
    que_corpus = []
    ans_corpus = []

    for so, tar in zip(src, tgt):
        src_tokens = tokenizer.morphs(preprocess_sentence(so))
        tgt_tokens = tokenizer.morphs(preprocess_sentence(tar))

        if (len(src_tokens) > 50): continue
        if (len(tgt_tokens) > 50): continue
    
        que_corpus.append(src_tokens)
        ans_corpus.append(tgt_tokens)

    return que_corpus, ans_corpus

In [11]:
que_corpus, ans_corpus = build_corpus(questions, answers, tokenizer)

In [12]:
print('que', que_corpus[:3],'+','ans', ans_corpus[:3])

que [['12', '시', '땡', '!'], ['1', '지망', '학교', '떨어졌', '어'], ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다']] + ans [['하루', '가', '또', '가', '네요', '.'], ['위로', '해', '드립니다', '.'], ['여행', '은', '언제나', '좋', '죠', '.']]


In [2]:
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import random
#wv = KeyedVectors.load("/home/aiffel-dj57/aiffel/chatbot/ko.bin")
model = Word2Vec.load('/home/aiffel-dj57/aiffel/chatbot/ko.bin')

#gensim을 3.8.1로 다운그레이드 한다.
#pip install gensim==3.8.1

In [3]:
#Lexical Substitution 구현
def lexical_sub(sentence, word2vec):
    import random

    res = ""
    toks = sentence.split()
    
    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[top][0]

    except:   # 단어장에 없는 단어
        return None

    for tok in toks:
        if tok is _from: res += _to + " "
        else: res += tok + " "

    return res

In [ ]:
#Augmentation된 que_corpus 와 원본 ans_corpus 가 병렬을 이루도록, 
#que_corpus 와 Augmentation된 ans_corpus 가 병렬을 이루도록 하여 전체 데이터가 원래의 3배가량으로 늘어나도록 합니다.
#데이터 30000개!

from tqdm import tqdm_notebook

new_corpus_qu = []
new_corpus_ans = []

for idx in tqdm_notebook(range(3000)):
    old_src = tokenizer.decode_ids(src_corpus[idx])

    new_src = lexical_sub(old_src, wv)

    if new_src is not None: new_corpus.append(new_src)

    new_corpus.append(old_src)

print(new_corpus[:10])